In [8]:
#need to import processed data

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

trainset = []
testset = []

with open("Datasets/19902014processedtraining.csv") as f:
  for row in f:
    trainset.append(row.strip("\n").split(","))

with open("Datasets/20202024processedtesting.csv") as f:
  for row in f:
    testset.append(row.strip("\n").split(","))

x_train = []
y_train = []
x_test = []
y_test = []

for record in trainset:
  newrec = []
  newrec = record[:5] + record[6:]
  x_train.append(newrec)
  y_train.append(record[5])

for record in testset:
  newrec = []
  if record != ['']:
    resold_price = record[5]
    newrec = record[:5] + record[6:]
    x_test.append(newrec)
    y_test.append(resold_price)

forest_model = RandomForestRegressor(n_estimators=100) #the number of trees that are being used

# Now fit the model on the training data
forest_model.fit(x_train, y_train)

from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y_test, forest_model.predict(x_test))
print("Mean Absolute Error:", mae)

# new_data = [[75, "4 ROOM", "2023-06", "85 years", 3.2, "Model A", 57000, 2.1]]  # Example row (needs to be encoded)

# prediction = forest_model.predict(new_data)
# print("Predicted resale price:", prediction[0])


ValueError: could not convert string to float: '9.2143047132.0'